## `Publisher` Data Dictionary (all features created by inner joining parts of `checkout_ph` and `collection` on `Publisher`)
* **publisher_key**: Publisher name, the table's primary key. 
* **pub_tot_checkouts**: The total amount of checkouts per publisher. Created by grouping by publisher and doing a `count()`.
* **pub_tot_books**: The total about of books per publisher. The total amount of checkouts per publisher. Created by grouping by publisher and doing a `CountDistinct()` on BibNum.
* **pub_checkout_percentile**: The percentile of a publisher in regards to their total checkouts. Created by using the `percent_rank()` on pub_tot_checkouts.
* **pub_books_percentile**: The percentile of a publisher in regards to their total books. Created by using the `percent_rank()` on pub_tot_books.

## Creating features by Publisher

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [0]:
# creating datatables from the database tables
collection = spark.table("library.collection")

checkout = spark.sql('''
SELECT *, MONTH(CheckoutDateTime) Month, YEAR(CheckoutDateTime) Year
FROM library.checkout_ph
'''
)

dictionary = spark.sql("SELECT * FROM library.data_dictionary").toPandas()

## filtering checkout data to sept 2017 to sept 2019

In [0]:
checkout.filter("((Year = 2017 and Month >= 9) or (Year >= 2018)) and ((Year = 2019 and Month <= 9) or (year < 2019))")

Out[8]: DataFrame[ID: string, CheckoutYear: bigint, BibNumber: bigint, ItemBarcode: string, ItemType: string, Collection: string, CallNumber: string, ItemTitle: string, Subjects: string, CheckoutDateTime: timestamp, Month: int, Year: int]

In [0]:
df_print_books = dictionary.query("code_type == 'ItemType' and format_subgroup == 'Book' and format_group == 'Print'")
book_codes = df_print_books.code.to_list()
print(book_codes)

# display(df_print_books)

['arbk', 'ucunknj', 'bcbk', 'jrbk', 'jcbk', 'acbk', 'jrmus', 'pkbknh', 'ucunkn', 'jcmus', 'ahbk']

In [0]:
books = collection \
  .withColumn('ReportDate', F.to_date(F.col("ReportDate"), "MM/dd/yyyy")) \
  .filter((F.col('ReportDate') >= F.lit("2017-09-01")) & (F.col('ReportDate') <= F.lit("2019-09-30")) & (F.col('ItemType').isin(book_codes)))
#display(books)

In [0]:
display(checkout)

ID,CheckoutYear,BibNumber,ItemBarcode,ItemType,Collection,CallNumber,ItemTitle,Subjects,CheckoutDateTime
201611051226000010077570322,2016,2866071,0010077570322,acbk,nanf,746.43204 V869 2012,Vogue Knitting the ultimate hat book history technique design,"Knitting Patterns, Hats",2016-11-05T12:26:00.000+0000
201608021407000010084202216,2016,3068717,0010084202216,jcbk,ncfic,J GURTLER,Shyannas wish,"Mermaids Juvenile fiction, Birthdays Juvenile fiction, Rescues Juvenile fiction, Friendship Juvenile fiction, Mermaids Fiction, Birthdays Fiction, Rescues Fiction, Friendship Fiction",2016-08-02T14:07:00.000+0000
201609251509000010086370375,2016,3086557,0010086370375,acbk,nafic,FIC RAEDER 2015,Black iris,Romantic suspense fiction,2016-09-25T15:09:00.000+0000
201601241512000010059048354,2016,3119021,0010059048354,acbk,nanf,641.5945 B2964L 2015,Lidias mastering the art of Italian cuisine everything you need to know to be a great Italian cook,"Cooking Italian, Cookbooks",2016-01-24T15:12:00.000+0000
201604291420000010087086442,2016,3172599,0010087086442,acbk,nanew,638.5789 G1676 2016,Gardening for butterflies how you can attract and protect beautiful beneficial insects,"Butterfly gardening, Butterflies",2016-04-29T14:20:00.000+0000
201609301327000010079972815,2016,2918837,0010079972815,acdvd,cadvd,DVD MUD,Mud,"Friendship Drama, Fugitives from justice Drama, Murder Drama, Islands Drama, Mississippi River Drama, Feature films, Coming of age films, Action and adventure films, Fiction films, Video recordings for the hearing impaired",2016-09-30T13:27:00.000+0000
201610021541000010084152221,2016,2663163,0010084152221,jcbk,ncrdr,ER ARNOLD,Fly Guy meets Fly Girl,Flies Juvenile fiction,2016-10-02T15:41:00.000+0000
201608061121000010072818825,2016,2718093,0010072818825,acbk,nanf,339.46091 B2235P 2011,Poor economics a radical rethinking of the way to fight global poverty,"Economic assistance Developing countries, Poverty Prevention",2016-08-06T11:21:00.000+0000
201701021402000010072412553,2017,2703609,0010072412553,acdvd,calndvd,HINDI DVD KITES,Kites,"Man woman relationships Drama, Jealousy Drama, Illegal aliens Drama, Las Vegas Nev Drama, Motion pictures Hindi, Fiction films, Feature films, Action and adventure films, Musical films",2017-01-02T14:02:00.000+0000
201604111202000010066691881,2016,2535614,0010066691881,jccd,cccd,CDJ 781.542 T494,Tinker Bell songs from and inspired by Disney fairies,"Motion picture music Juvenile sound recordings, Fairies Songs and music Juvenile sound recordings, Childrens songs Juvenile sound recordings, Motion picture music, Fairies Songs and music, Songs",2016-04-11T12:02:00.000+0000


In [0]:
display(books)

BibNum,Title,Author,ISBN,PublicationYear,Publisher,Subjects,ItemType,ItemCollection,FloatingItem,ItemLocation,ReportDate,ItemCount
606389,"Smithville Lake, Missouri.",null,null,[1986],"US Army Corps of Engineers, Kansas City District,","Outdoor recreation Missouri Smithville Lake, Smithville Lake Mo Recreational use",acbk,canf,null,cen,2019-07-01,1
3278819,My little book of tractors / by Rod Green.,"Green, Rod (Journalist)","1682971538, 9781682971536",[2017],"QEB,",Tractors Juvenile literature,jcbk,ncenf,null,bea,2019-07-01,1
3451550,"The Parisian, or, Al-Barisi : a novel / Isabella Hammad.","Hammad, Isabella.","0802129439, 9780802129437",2019.,"Grove Press,","Autonomy and independence movements Fiction, Middle East History Fiction, Nablus Fiction, Paris France Fiction, Bildungsromans, Historical fiction",acbk,nafic,null,wts,2019-07-01,1
3013518,My heart is laughing / written by Rose Lagercrantz ; illustrated by Eva Eriksson.,"Lagercrantz, Rose, 1947-","1877579513, 1877579521, 9781877579516, 9781877579523",2014.,"Gecko Press,","Bullying Juvenile fiction, Happiness Juvenile fiction, Schools Juvenile fiction",jcbk,ncef,null,swt,2019-07-01,1
2729577,"All about acrylics : a complete guide to painting using this versatile medium / Oliver Löhr, Kristina Schaper and Ute Zander; with photographs by Stefan Boekels.","Löhr, Oliver","1844486052, 9781844486052",2011.,"Search,",Acrylic painting Technique,acbk,nanf,null,nga,2019-07-01,1
2801859,The Obamians : the struggle inside the White House to redefined American power / James Mann.,"Mann, Jim, 1946-","0670023760, 9780670023769",2012.,"Viking,","Obama Barack, Obama Barack Political and social views, Obama Barack Friends and associates, Democratic Party U S History 21st century, Power Social sciences United States History 21st century, Presidents United States Biography, Politicians United States Biography, United States Foreign relations 2009, United States Military policy, United States Politics and government 2009",acbk,canf,null,cen,2019-07-01,1
3102735,"Sensation Comics featuring Wonder Woman. Volume 1 / Gail Simone (and 10 others), writers ; Ethan Van Sciver (and 10 others), artists ; Brian Miller of Hi-Fi Colour Design (and 7 others), colorists ; Saida Temofonte, Deron Bennett, letterers ; Gene Ha Collection cover artist.",null,"140125344X, 9781401253448",[2015],"DC Comics,","Wonder Woman Fictitious character Comic books strips etc, Women superheroes Comic books strips etc, Comic books strips etc United States, Graphic novels, Superhero comic books strips etc",acbk,nycomic,null,bal,2019-07-01,1
730418,Final report of the Ad Hoc Committee for a Central Area School Council / [Seattle Public Schools].,Ad Hoc Committee for a Central Area School Council.,null,[1969],"[The Committee],","Seattle Public Schools, Public schools Washington State Seattle Citizen participation, Community organization Washington State Seattle, Community and school Washington State Seattle, Citizens advisory committees in education Washington State Seattle, Central District Seattle Wash",arbk,casea,null,cen,2019-07-01,1
1709321,Some smug slug / by Pamela Duncan Edwards ; illustrated by Henry Cole.,"Edwards, Pamela Duncan","0060247894, 0060247924",[1996],"HarperCollinsPublishers,","Slugs Mollusks Juvenile fiction, Animals Fiction",jcbk,ncpic,null,net,2019-07-01,1
2692931,"A joyful Christmas : a treasury of new and classic songs, poems, and stories for the holiday / collected and illustrated by James Ransome.",null,"0805066217, 9780805066210",2010.,"Henry Holt,","Christmas Literary collections, American literature 21st century, American literature Juvenile literature",jcbk,nchol,null,swt,2019-07-01,1


In [0]:
checkout_join = checkout.select(checkout.BibNumber)

## creating my dataset with publisher to create features

In [0]:
# creating my dataset with publisher to create features
pub = books.join(checkout_join, checkout_join.BibNumber ==  books.BibNum, "inner") 

pub = pub.select(["BibNum", "Publisher", "PublicationYear", "Title"])

In [0]:
display(pub)

BibNum,Publisher,PublicationYear,Title
65,"McGraw-Hill,",[1968],The continuing revolution; a history of physics from the Greeks to Einstein
65,"McGraw-Hill,",[1968],The continuing revolution; a history of physics from the Greeks to Einstein
65,"McGraw-Hill,",[1968],The continuing revolution; a history of physics from the Greeks to Einstein
65,"McGraw-Hill,",[1968],The continuing revolution; a history of physics from the Greeks to Einstein
65,"McGraw-Hill,",[1968],The continuing revolution; a history of physics from the Greeks to Einstein
65,"McGraw-Hill,",[1968],The continuing revolution; a history of physics from the Greeks to Einstein
65,"McGraw-Hill,",[1968],The continuing revolution; a history of physics from the Greeks to Einstein
65,"McGraw-Hill,",[1968],The continuing revolution; a history of physics from the Greeks to Einstein
65,"McGraw-Hill,",[1968],The continuing revolution; a history of physics from the Greeks to Einstein
65,"McGraw-Hill,",[1968],The continuing revolution; a history of physics from the Greeks to Einstein


## creating a total checkouts by publisher in the SPL system column

In [0]:
# creating a total checkouts by publisher feature
chkts_per_pub = pub.groupby("Publisher").count()

In [0]:
chkts_per_pub.show()

+--------------------+---------+
 Publisher| count|
+--------------------+---------+
 Dodd, Mead,| 162578|
 Lakeside Press,| 1960|
Congress on Resea...| 172|
 Putnam's,| 7571909|
British Library a...| 621|
Hyperion Books fo...|222116902|
Greenwillow Books...| 16901482|
 Crown Pub.,| 1797101|
Design Originals,...| 63962|
 Namaste Pub.,| 102971|
Chronicle Books LLC,| 8365513|
Forever, an impri...| 1176|
 Hisakatachairudo,| 8109|
Simon Spotlight, ...| 34529|
Book Division, Fa...| 18|
 Doubleday| 4034836|
 Avery Pub. Group,| 1641|
 Phaidon Press,| 1060167|
N. Yeh's Art Studio,| 2107|
Andrews McMeel Pub.,| 8924712|
+--------------------+---------+
only showing top 20 rows

##creating a total unique bib numbers by publisher in the SPL system column

In [0]:
# creating total books by publisher
books_per_pub = books.groupBy("Publisher").agg(countDistinct(col("BibNum")))

## joining the two columns

In [0]:
# joining features into main dataset

publisher = chkts_per_pub.join(books_per_pub, ["Publisher"], "outer")

In [0]:
publisher = publisher.withColumnRenamed("count", "total_checkouts")
publisher = publisher.withColumnRenamed("count(BibNum)", "total_books")

In [0]:
# replacing null's with 0's
publisher = publisher.na.fill(value=0)

## creating publisher rank in system in terms of checkout totals

In [0]:
windowSpec = Window \
.partitionBy() \
.orderBy(col("total_checkouts"))

publisher = publisher \
.withColumn("chkts_percent_rank", percent_rank().over(windowSpec))

## creating publisher rank in system in terms of bibnumber totals

In [0]:
windowSpec = Window \
.partitionBy() \
.orderBy(col("total_books"))



publisher = publisher \
.withColumn("books_percent_rank", percent_rank().over(windowSpec))

In [0]:
publisher = publisher.withColumnRenamed("total_checkouts", "pub_tot_checkouts")
publisher = publisher.withColumnRenamed("total_books", "pub_tot_books")
publisher = publisher.withColumnRenamed("chkts_percent_rank", "pub_checkout_percentile")
publisher = publisher.withColumnRenamed("books_percent_rank", "pub_books_percentile")

In [0]:
publisher = publisher.withColumnRenamed("Publisher", "publisher_key")

In [0]:
display(publisher)

Publisher,pub_tot_checkouts,pub_tot_books,pub_checkout_percentile,pub_books_percentile
null,857735,0,0.9926135858670643,0.0
"""The Stage,""",0,1,0.0,9.874885204459498E-6
"""The Studio,"",",0,1,0.0,9.874885204459498E-6
"""The Times"",",0,1,0.0,9.874885204459498E-6
"1797 House,",0,1,0.0,9.874885204459498E-6
"A. Burdick,",0,1,0.0,9.874885204459498E-6
"A. Constable and Company, limited,",0,1,0.0,9.874885204459498E-6
A. Hatier,0,1,0.0,9.874885204459498E-6
"A. L. Bancroft and Company,",0,1,0.0,9.874885204459498E-6
A. Morancé.,0,1,0.0,9.874885204459498E-6


## creating a `publisher` table in `library_features` and populating it with `publisher` data

In [0]:
publisher.write.saveAsTable("library_features.publisher")